In [17]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16

In [18]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import os
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# Path to original dataset
data_dir = '/content/drive/MyDrive/images_money_dataset'

# Path to save augmented images
aug_dir = '/content/GG'

# Create augmented dataset directory if it doesn't exist
if not os.path.exists(aug_dir):
    os.makedirs(aug_dir)


In [21]:
for class_folder in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_folder)
    aug_class_path = os.path.join(aug_dir, class_folder)

    # Create class directory in augmented dataset folder if it doesn't exist
    if not os.path.exists(aug_class_path):
        os.makedirs(aug_class_path)

    for filename in os.listdir(class_path):
        img_path = os.path.join(class_path, filename)
        img = load_img(img_path)  # Load image
        x = img_to_array(img)  # Convert image to numpy array
        x = x.reshape((1,) + x.shape)  # Reshape it for the generator

        i = 0
        # Generate 5 augmented images for each input image
        for batch in datagen.flow(x, batch_size=1, save_to_dir=aug_class_path, save_prefix='aug', save_format='jpeg'):
            i += 1
            if i > 10:
                break  # Stop after generating 5 images


In [22]:
SIZE = 256  #Resize images

#Capture training data and labels into respective lists
data_images = []
data_labels = []

for directory_path in glob.glob("/content/GG/*"):
    label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpeg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        data_images.append(img)
        data_labels.append(label)


#  Split Data

In [23]:

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(data_images, data_labels, test_size=0.2, random_state=42, stratify=data_labels)


In [24]:
# prompt: convert train tesst to nummpy array

X_train = np.array(X_train)
X_test = np.array(X_test)

train_labels = np.array(y_train)
test_labels = np.array(y_test)
X_train.shape

(1922, 256, 256, 3)

In [25]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_labels)
y_train1 = le.transform(train_labels)
le.fit(test_labels)
y_test1 = le.transform(test_labels)

In [26]:
X_train,X_test  = X_train / 255.0, X_test / 255.0

In [27]:
VGG_model = VGG16(weights="imagenet", include_top=False, input_shape=(SIZE, SIZE,3))

In [28]:
for layer in VGG_model.layers:
	layer.trainable = False

VGG_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [54]:
VGG_model.save('vgg_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:

feature_extractor=VGG_model.predict(X_train)

features = feature_extractor.reshape(feature_extractor.shape[0], -1)

X_for_training = features

from sklearn.neural_network import MLPClassifier

# Create an MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(200,), activation='relu', solver='adam', max_iter=2500)

# Fit the MLP classifier to the training data
mlp.fit(X_for_training, y_train1)

X_test_feature = VGG_model.predict(X_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

# Predict the labels for the test data
y_pred_mlp = mlp.predict(X_test_features)

# Calculate the accuracy of the MLP classifier
accuracy_mlp = mlp.score(X_test_features, y_test1)

# Print the accuracy
print("Accuracy of the MLP classifier:", accuracy_mlp)


16/16 [==============================] - 348s 22s/step
Accuracy of the MLP classifier: 0.9230769230769231


In [51]:
import joblib

# Assuming your MLP classifier is named 'mlp'
joblib.dump(mlp, 'mlp_classifier_model.pkl')


['mlp_classifier_model.pkl']

In [30]:

from sklearn.metrics import precision_score, recall_score, confusion_matrix

# Calculate precision and recall
precision = precision_score(y_test1, y_pred_mlp, average='weighted')
recall = recall_score(y_test1, y_pred_mlp, average='weighted')

# Calculate confusion matrix
confusion_matrix = confusion_matrix(y_test1, y_pred_mlp)

# Print the results
print("Precision:", precision)
print("Recall:", recall)
print("Confusion Matrix:\n", confusion_matrix)


Precision: 0.9257979851915777
Recall: 0.9230769230769231
Confusion Matrix:
 [[18  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 13  1  1  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  2 36  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0]
 [ 1  0  0 25  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0 12  2  0  0  0  0  0  0  2  0  0  0  0  0  0  0]
 [ 1  1  1  0  2 17  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0 37  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 22  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 22  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 18  0  0  0  1  0  0  0  3  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 21  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  1  0  0  0  0 25  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 21  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0  0  1  0  0  

In [49]:


from sklearn.neighbors import KNeighborsClassifier

# Create a KNN classifier with 3 neighbors
knn = KNeighborsClassifier(n_neighbors=3)

# Fit the KNN classifier to the training data
knn.fit(X_for_training, y_train1)

# Predict the labels for the test data
y_pred_knn = knn.predict(X_test_features)

# Calculate the accuracy of the KNN classifier
accuracy_knn = knn.score(X_test_features, y_test1)

# Print the accuracy
print("Accuracy of the KNN classifier:", accuracy_knn)

# Calculate precision and recall
precision = precision_score(y_test1, y_pred_knn, average='weighted')
recall = recall_score(y_test1, y_pred_knn, average='weighted')


# Print the results
print("Precision:", precision)
print("Recall:", recall)



#Check results on a few select images
n=np.random.randint(0, X_test.shape[0])
img = X_test[n]
input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
input_img_feature=VGG_model.predict(input_img)
input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
prediction = knn.predict(input_img_features)[0]
prediction = le.inverse_transform([prediction])  #Reverse the label encoder to original name
print("The prediction for this image is: ", prediction)
print("The actual label for this image is: ", test_labels[n])


Accuracy of the KNN classifier: 0.6528066528066528
Precision: 0.7029917614517565
Recall: 0.6528066528066528
1/1 [==============================] - 1s 628ms/step
The prediction for this image is:  ['/content/GG/R50_gen_b1']
The actual label for this image is:  /content/GG/R50_gen_b1


In [50]:


from sklearn import tree

# Create a decision tree classifier
dtree = tree.DecisionTreeClassifier()

# Fit the decision tree classifier to the training data
dtree.fit(X_for_training, y_train1)

# Predict the labels for the test data
y_pred_dtree = dtree.predict(X_test_features)

# Calculate the accuracy of the decision tree classifier
accuracy_dtree = dtree.score(X_test_features, y_test1)

# Calculate precision and recall
precision_dtree = precision_score(y_test1, y_pred_dtree, average='weighted')
recall_dtree = recall_score(y_test1, y_pred_dtree, average='weighted')


# Print the results
print("Accuracy of the decision tree classifier:", accuracy_dtree)
print("Precision of the decision tree classifier:", precision_dtree)
print("Recall of the decision tree classifier:", recall_dtree)



Accuracy of the decision tree classifier: 0.3264033264033264
Precision of the decision tree classifier: 0.3328287397100514
Recall of the decision tree classifier: 0.3264033264033264
